# 객체 참조, 가변성 재활용

- 파이썬 변수를 은유적으로 표현
- 객체의 정체성, 동질성, 별명
- 튜플은 불변형이지만 그 안에 들어 있는 값은 바뀔 수 있다.
- 얕은 복사와 깊은 복사
- 가비지 컬렉션과 del 명령 및 객체를 보존하지 않으면서 객체를 '기억'하기 위한 약한 참조

## == 연산자와 is 연산자 간의 선택
== 연산자가 객체의 값을 비교하는 반면, is 연산자는 객체의 정체성을 비교한다.
싱글턴과 비교할 때는 is 연산자를 사용해야한다.
is 연산자는 오버로딩 할 수 없으므로 파이썬이 이 값을 평가하기 위해 특별 메서드를 호출할 필요가 없고, 두 정수를 비교하는 정도로
연산이 간단하므로 is 연산자가 == 연산자보다 빠르다.

## 기본 복사는 얕은 복사
리스트나 대부분의 내장 가변 컬렉션을 복사하는 가장 손쉬운 방법은 그 자료형 자체의 내장 생성자를 사용하는 것이다.

In [2]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
print(l2)
print(l2 == l1)
print(l2 is l1)

[3, [55, 44], (7, 8, 9)]
True
False


## 객체의 깊은 복사와 얕은 복사
copy 모듈이 제공하는 deepcopy() 함수는 깊은 복사를, copy()함수는 얕은 복사를 지원한다.

## 참조로서의 함수 매개변수
파이썬은 공유로 호출하는 매개변수 전달 방식만 지원한다. 이 방식은 루비, 스몰토크, 자바 등 대부분의 객체지향 언어에서 사용하는 방식과 동일하다.
달리 말하면 함수 안의 매개변수는 실제 인수의 별명이 된다.

## 가변형을 매개변수 기본값으로 사용하기: 좋지 않은 생각
기본값을 가진 선택적 인수는 파이썬 함수 정의에서 아주 좋은 기능으로, 하위 호환성을 유지하며 API를 개선할 수 있게 해준다.
그러나 매개변수 기본값으로 가변 객체를 사용 하는 것은 피해야 한다.

## del과 가비지 컬렉션
`del` 명령은 이름을 제거하는 것이지 객체를 제거하는 것이 아니다. del 명령어의 결과로 객체가 가비지컬렉트 될 수 있지만.
제거된 변수가 객체를 참조하는 최후의 변수거나 객체에 도달할 수 없으 때만 가비지 컬렉트 된다.

CPython의 경우 가비지 컬렉션은 주로 참조 카운트에 기반한다. 본질적으로 각 객체는 얼마나 많은 참조가 자신을 가리키는지 개수를 세고 있다.

## WeakValueDictionary 촌극
`WeakValueDictionary` 클래스는 객체에 대한 약한 참조를 값으로 가지는 가변 매핑을 구현한다.
참조된 객체가 프로그램 다른 곳에서 가비지 컬랙트 되면 해당 키도 `WeakValueDictionary`에서 자동으로 제거된다.
이 클래스는 일반적으로 캐시를 구현하기 위해 사용된다.

In [3]:
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [7]:
import weakref

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese

print(sorted(stock.keys()))
del catalog
# 임시 변수가 객체를 참조함으로써 예상보다 객체의 수명이 늘어날 수 있다. 지역 변수는 함수가 반환되면서 사라지므로 일반적으로 문제가 되지 않는다.
# 그러나 for 루프 변수인 cheese는 전역 변수 이므로 명시적으로 제거하기 전에는 사라지지 않는다.
print(sorted(stock.keys()))
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
[]


CPython은 물론 모든 파이썬 인터프리터에서 파일 연산은 with문을 사용하는 것이 좋다. with문은 팡리이 열려 있는 동안 예외가 발생해도 파일을 반드시 닫는다.

In [ ]:
with open('test.txt', 'wt', encoding='utf-8') as fp:
    fp.write('1, 2, 3')